<a href="https://colab.research.google.com/github/MShiloni22/DDBMS_Project_A/blob/master/DDBMS_Project_A.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install pyspark
!pip install findspark

     |████████████████████████████████| 281.4 MB 31 kB/s 
     |████████████████████████████████| 198 kB 36.9 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=5d2bf277316f09bb2b82d58d6c0ffc3121c03d81f95f07c8398b677f7d1c18ca
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [153]:
import os
import findspark
import datetime as dt

findspark.init()
from pyspark.sql import SparkSession
def init_spark(app_name: str):
 spark = SparkSession.builder.appName(app_name).getOrCreate()
 sc = spark.sparkContext
 return spark, sc
spark, sc = init_spark('demo')
sc

<SparkContext master=local[*] appName=demo>

In [154]:
# working on queries.csv extraction
from pyspark.sql import SparkSession,Row, Column
import pyspark.sql.functions as F
spark = SparkSession.builder.appName('SparkByExamples.com').getOrCreate()
queries_file = '/content/drive/MyDrive/Colab Files/DDBMS//queries.csv'
df = spark.read.csv(queries_file, header='True', inferSchema='True')

print("Loaded queries successfully")


Loaded queries successfully


In [155]:
# doing the same process for all columns
column_names = ["genres", "lang", "actors", "director", "cities", "country", 
                "from_realese_date", "production_company"]
for name in column_names:
  temp_name_1 = name + "1"
  temp_name_2 = name + "2"
  # removing irrelevant chars
  df = df.select("*", F.translate(F.col(name), "'[]", "")\
                .alias(temp_name_1))\
  .drop(name)

  # converting arrays strings to arrays of strings
  df = df.select("*", F.split(F.col(temp_name_1),",").alias(temp_name_2)) \
      .drop(temp_name_1)
  df = df.withColumnRenamed(temp_name_2,name)

print("Cleaned and seperated to arrays")

Cleaned and seperated to arrays


In [156]:
# creating columns for each element in each column contains an array
for name in column_names:
  name_size = name + "_size"
  # get max array's size in the column
  df = df.withColumn(name_size, F.size(F.col(name)))
  df_col_max = df.agg({name_size: 'max'})
  max_col_size = df_col_max.collect()[0][0]

  for i in range(max_col_size):
    df = df.withColumn(name + "_" + str(i), F.col(name)[i])

  df = df.drop(name)
  df = df.drop(name_size)


def drop_null_columns(df):
  
    """
    This function drops columns containing all null values.
    :param df: A PySpark DataFrame
    """
    
    null_counts = df.select([F.count(F.when(F.col(c).isNull(), c)).alias(c) for c in df.columns]).collect()[0].asDict()
    to_drop = [k for k, v in null_counts.items() if v >= df.count()]
    df = df.drop(*to_drop)
    
    return df

df = drop_null_columns(df)
df.show()

+-------+---------+----------------+----------------+-------+---------+------+------------------+----------------+--------------------+---------------+----------------+------------+----------+--------------+----------+--------+-----------+----------------+---------+-------------------+-------------------+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|user_id| genres_0|        genres_1|        genres_2| lang_0|   lang_1|lang_2|          actors_0|        actors_1|            actors_2|       actors_3|      director_0|  director_1|director_2|      cities_0|  cities_1|cities_2|  country_0|       country_1|country_2|from_realese_date_0|from_realese_date_1|from_realese_date_2|production_company_0|production_company_1|production_company_2|production_company_3|production_company_4|
+-------+---------+----------------+----------------+-------+---------+------+------------------+----------------+--------------------+-